### Making Pauli string from an Operator converting it to Matrix and then using In Built Function

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from qiskit.quantum_info import SparsePauliOp

ns = 3
numq = 2*ns
t = 1.0
U = 2.0

X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.array([[1,0],[0,1]])


string = numq*[I]
C = []
for i in range(numq):
    string[0:i] = i*[Z]
    string[i] = 0.5*(X + 1j*Y)
    total = string[0]
    for j in range(1,numq):
        total = np.kron(total,string[j])

    C.append(total)

Hopp = np.zeros([2**numq,2**numq])
Self = np.zeros([2**numq,2**numq])

for j in range(ns-1):
    Hopp = Hopp + C[j].T.conj()@C[j+1] + C[j+1].T.conj()@C[j]
    Hopp = Hopp + C[j+ns].T.conj()@C[j+1+ns] + C[j+1+ns].T.conj()@C[j+ns]

for j in range(ns):
    Self = Self + C[j].T.conj()@C[j]@C[j+ns].T.conj()@C[j+ns]
    
Hamilton = -t*Hopp + U*Self
print(Hamilton.real)

pauli_s3 = SparsePauliOp.from_operator(Hamilton)

print(pauli_s3)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0. -1. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  4. -1.  0.]
 [ 0.  0.  0. ... -1.  4.  0.]
 [ 0.  0.  0. ...  0.  0.  6.]]
SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIXX', 'IIIIYY', 'IIIIZI', 'IIIXXI', 'IIIYYI', 'IIIZII', 'IIZIII', 'IIZIIZ', 'IXXIII', 'IYYIII', 'IZIIII', 'IZIIZI', 'XXIIII', 'YYIIII', 'ZIIIII', 'ZIIZII'],
              coeffs=[ 1.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,
 -0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j,
 -0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j])


### Direct from J-W Transformation

In [7]:
import sympy as sp
from sympy.physics.quantum import TensorProduct
from sympy.physics.paulialgebra import Pauli,evaluate_pauli_product
import numpy as np

ns = 3
numq = 2*ns

X,Y,Z = Pauli(1),Pauli(2),Pauli(3)
Iden = Pauli(1)*Pauli(1)

C,Cd = [],[]

# Converting all the creation and annihilation operators to corresponding Pauli strings
for j in range(numq):
    pC = numq*[Iden] 
    pC[0:j] = j*[Z] 
    pC[j]  = (X+1j*Y)*0.5
    pCd = pC.copy()
    pCd[j] = (X-1j*Y)*0.5
    C.append(pC)
    Cd.append(pCd)
    
def mult(*operator):  
    pearl = np.array(operator[0])
    for kk in range(1,len(operator)):
        pearl = pearl*np.array(operator[kk])
    emp = pearl[0]
    for item in range(1,len(pearl)):
        emp = TensorProduct(emp,pearl[item])     
    lott = evaluate_pauli_product(emp.expand(tensorproduct=True))
    return lott

# print(Cd[0],C[1])
# logo = mult(Cd[4],C[4],Cd[0],C[0]) 

Hopp,Self = evaluate_pauli_product(0),evaluate_pauli_product(0)

for j in range(ns-1):
    Hopp = Hopp + mult(Cd[j],C[j+1]) + mult(Cd[j+1],C[j]) + mult(Cd[j+ns],C[j+1+ns]) + mult(Cd[j+1+ns],C[j+ns])

for j in range(ns):
    Self = Self + mult(Cd[j],C[j],Cd[j+ns],C[j+ns])
    
Hamilton = -t*Hopp + U*Self
result = Hamilton.expand(tensorproduct = True)
result

1.5*1x1x1x1x1x1 - 0.5*1x1x1x1x1xsigma3 - 0.5*1x1x1x1xsigma1xsigma1 - 0.5*1x1x1x1xsigma2xsigma2 - 0.5*1x1x1x1xsigma3x1 - 0.5*1x1x1xsigma1xsigma1x1 - 0.5*1x1x1xsigma2xsigma2x1 - 0.5*1x1x1xsigma3x1x1 - 0.5*1x1xsigma3x1x1x1 + 0.5*1x1xsigma3x1x1xsigma3 - 0.5*1xsigma1xsigma1x1x1x1 - 0.5*1xsigma2xsigma2x1x1x1 - 0.5*1xsigma3x1x1x1x1 + 0.5*1xsigma3x1x1xsigma3x1 - 0.5*sigma1xsigma1x1x1x1x1 - 0.5*sigma2xsigma2x1x1x1x1 - 0.5*sigma3x1x1x1x1x1 + 0.5*sigma3x1x1xsigma3x1x1

In [12]:
sub_dict = {"sigma1":"X","sigma2":"Y","sigma3":"Z","x":""}
modified = str(result)
for elem in sub_dict:
    modified = modified.replace(elem,sub_dict[elem])
modified

'1.5*111111 - 0.5*11111Z - 0.5*1111XX - 0.5*1111YY - 0.5*1111Z1 - 0.5*111XX1 - 0.5*111YY1 - 0.5*111Z11 - 0.5*11Z111 + 0.5*11Z11Z - 0.5*1XX111 - 0.5*1YY111 - 0.5*1Z1111 + 0.5*1Z11Z1 - 0.5*XX1111 - 0.5*YY1111 - 0.5*Z11111 + 0.5*Z11Z11'

In [14]:
pauli_s3

SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIXX', 'IIIIYY', 'IIIIZI', 'IIIXXI', 'IIIYYI', 'IIIZII', 'IIZIII', 'IIZIIZ', 'IXXIII', 'IYYIII', 'IZIIII', 'IZIIZI', 'XXIIII', 'YYIIII', 'ZIIIII', 'ZIIZII'],
              coeffs=[ 1.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,
 -0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j,
 -0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j])